# SKKU TNT 2019-2

## Programming Exercise 2: Linear regression with multiple variables

In [3]:
# 디렉토리 경로 설정에 사용되는 패키지
import os

# 벡터 계산
import numpy as np
import pandas as pd

# 시각화 라이브러비
from matplotlib import pyplot
from mpl_toolkits.mplot3d import Axes3D  # needed to plot 3-D surfaces

# 과제 제출을 위한 라이브러리
import utils 

# 이 연습에 대한 제출, 채점 라이브러리
grader = utils.Grader()

# matplotlib가 노트북에 플롯을 포함하도록 지시
%matplotlib inline

지난주 자료를 성공적으로 완성했다면 TNT 코드1팀이 앤드류 응 교수님 대신 칭찬해 드리도록 하죠! 이제 당신은 선형 회귀를 이해했고 자신의 데이터 셋에서 선형 회귀를 사용할 수 있습니다.

이 프로그래밍 연습의 나머지 부분에서는 다음과 같은 선택적 연습이 포함되었습니다. 이 연습은 자료에 대한 심층적인 이해를 돕기 때문에 가능하면 이를 해보는 것이 좋습니다. 이 연습에 대한 답안을 제출하고 제촐한 답안이 올바른지 확인할 수 있습니다.


## 3 Linear regression with multiple variables

이 부분에서는 주택의 가격을 예측하기 위해 여러 변수로 선형 회귀를 구현합니다. 집을 팔고 있고 좋은 시장 가격이 얼마인지 알고 싶다고 가정 해 봅시다. 이를 수행하는 한 가지 방법은 먼저 판매 된 최근 주택에 대한 정보를 수집하고 주택 가격 모델을 만드는 것입니다.

`Data / ex1data2.txt` 파일에는 오리건 주 포틀랜드에있는 주택 가격에 대한 훈련 세트가 포함되어 있습니다. 첫 번째 열은 집의 크기 (n square feet), 두 번째 열은 침실 수, 세 번째 열은 가격입니다.

<a id="section4"></a>

### 3.1 Feature Normalization

이 데이터 세트에서 일부 값을로드하고 표시하는 것으로 시작합시다. 값을 살펴보면 집 크기는 침실 수의 약 1000 배입니다. Feature가 차수에 따라 다르면, 처음 수행하는 Feature scaling은 gradient descent를 훨씬 더 빠르게 수렴시킬 수 있습니다.

In [4]:
# data 불러오기
data = np.loadtxt(os.path.join('Data', 'ex1data2.txt'), delimiter=',')
X = data[:, :2]
y = data[:, 2]
m = y.size

pd.DataFrame({'X1':X[:,0], 'X2':X[:,1], 'y':y}).iloc[:10]

,X1,X2,y
0,2104.0,3.0,399900.0
1,1600.0,3.0,329900.0
2,2400.0,3.0,369000.0
3,1416.0,2.0,232000.0
4,3000.0,4.0,539900.0
5,1985.0,4.0,299900.0
6,1534.0,3.0,314900.0
7,1427.0,3.0,198999.0
8,1380.0,3.0,212000.0
9,1494.0,3.0,242500.0


`featureNormalize` 함수에서 코드를 완성해봅시다.

* dataset에서 각 feature의 평균값을 뺍니다.
* 평균을 뺀 후 feature를 각각의 표준 편차로 스케일링(나누기)합니다.

표준 편차는 특정 피쳐의 값 범위에 얼마나 많은 변화가 있는지를 측정하는 방법입니다 (대부분의 데이터 포인트는 평균의 ± 2 표준 편차 내에 있음). 이것은 값의 범위 (max-min)를 대신하는 대안입니다. `numpy`에서`std` 함수를 사용하여 표준 편차를 계산할 수 있습니다.

예를 들어, $X [:, 0]$ 에는 트레이닝 세트 $ x_1 $ (집 크기)의 모든 값이 포함되므로 `np.std (X [:, 0])`는 집크기 feature의 표준 편차를 계산합니다.

함수 featureNormalize가 호출 될 때 $ x_0 = 1 $에 해당하는 1의 추가 열이 아직 $ X $에 추가되지 않았습니다.


<div class="alert alert-block alert-warning">
**Implementation Note:**
기능을 정규화 할 때는 정규화에 사용 된 값 (평균값 및 계산에 사용 된 표준 편차)을 저장하는 것이 중요합니다. 모델에서 매개 변수를 학습 한 후 종종 우리가 전에 보지 못한 주택의 가격을 예측하려고 합니다. 새로운 x 값 (거실 공간 및 침실 수)이 주어지면 먼저 훈련 세트에서 이전에 계산 한 평균 및 표준 편차를 사용하여 x를 정규화해야합니다.
</div>
<a id="featureNormalize"></a>

In [9]:
# 평균 및 표준편차 구하기
# 직접 작성해주세요.
mu = None # X의 평균
std = None # X의 표준편차

print(mu)
print(mu.shape)
print(std)
print(std.shape)

[2000.68085106    3.17021277]
(2,)
[7.86202619e+02 7.52842809e-01]
(2,)


In [ ]:
def  featureNormalize(X):
    """
    X의 피처를 정규화합니다.
    각 피처의 평균값이 0이고 표준 편차가 1 인 정규화 된 X 버전을 반환합니다.
    이는 종종 학습 알고리즘으로 작업 할 때 수행하는, 괜찮은 전처리 단계입니다.
    
    [Parameters]
    ----------
    X : array 형태 (m x n).
    
    [Returns]
    -------
    X_norm : 정규화된 data set의 array 형태 (m x n)
    
    [Instructions]
    ------------
    먼저 각 피처 차원에 대해 피처의 평균을 계산하고
    데이터 집합에서 빼고 평균 값을 mu로 저장합니다.
    그런 다음 각 피처의 표준 편차를 계산하고
    각 피처를 표준 편차로 나누고 표준 편차를 시그마로 저장합니다.
    
    X는 각 열이 피처이고 각 행이 예인 행렬입니다.
    각 기능에 대해 표준화를 별도로 수행해야합니다.
    
    [Hint]
    ----
    'np.mean'및 'np.std'기능 사용을 권장합니다.
    """
    
    # 아래 값을 올바르게 설정해야합니다
    # =========================== YOUR CODE HERE =====================
    mu = None
    sigma = None
    X_norm = None
    # ================================================================
    return X_norm, mu, sigma

구현 된`featureNormalize` 기능을 실행하려면 다음 셀을 실행하십시오.

In [ ]:
# 로드 된 데이터에서 "featureNormalize"호출
X_norm, mu, sigma = featureNormalize(X)

print('Computed mean:', mu)
print('Computed standard deviation:', sigma)

*정답을 제출하세요*

In [ ]:
grader.answer[0] = featureNormalize
grader.grade(0)

`featureNormalize` 함수가 테스트 된 후, 이제 우리는 절편 항(상수항)을`X_norm`에 추가합니다 :

In [17]:
# X에 절편 항 추가하기
X = np.concatenate([np.ones((m, 1)), X_norm], axis=1)
X[:10]

array([[1.000e+00, 1.000e+00, 2.104e+03, 3.000e+00],
       [1.000e+00, 1.000e+00, 1.600e+03, 3.000e+00],
       [1.000e+00, 1.000e+00, 2.400e+03, 3.000e+00],
       [1.000e+00, 1.000e+00, 1.416e+03, 2.000e+00],
       [1.000e+00, 1.000e+00, 3.000e+03, 4.000e+00],
       [1.000e+00, 1.000e+00, 1.985e+03, 4.000e+00],
       [1.000e+00, 1.000e+00, 1.534e+03, 3.000e+00],
       [1.000e+00, 1.000e+00, 1.427e+03, 3.000e+00],
       [1.000e+00, 1.000e+00, 1.380e+03, 3.000e+00],
       [1.000e+00, 1.000e+00, 1.494e+03, 3.000e+00]])

<a id="section5"></a>
### Gradient Descent (경사하강)

이전에는 일 변량 회귀 문제에 대해 경사 하강을 구현했습니다. 유일한 차이점은 $ X $ 매트릭스에 하나 이상의 기능이 있다는 것입니다. 가설 함수 및 배치 경사 하강에 대한업데이트 규칙은 변경되지 않습니다.

여러 변수를 가진 선형 회귀에 대한 비용 함수와 경사 하강을 구현하려면 'computeCostMulti'및 'gradientDescentMulti'함수의 코드를 완성해야합니다. 이전 부분의 코드 (단일 변수)가 이미 여러 변수를 지원하는 경우 여기에서도 사용할 수 있습니다.

코드가 여러 기능을 지원하는지, 그리고 정상적으로 벡터화되어 있는지 확인하십시오.
`numpy` 배열의`shape` 속성을 사용하여 데이터 셋에 몇 개의 기능이 있는지 알아낼 수 있습니다.

<div class="alert alert-block alert-warning">
**Implementation Note:** 다변량 경우 비용 함수는 다음과 같은 벡터화 된 형태로 작성 될 수 있습니다.

$$ J(\theta) = \frac{1}{2m}(X\theta - \vec{y})^T(X\theta - \vec{y}) $$

다음과 같을 때!

$$ X = \begin{pmatrix}
          - (x^{(1)})^T - \\
          - (x^{(2)})^T - \\
          \vdots \\
          - (x^{(m)})^T - \\ \\
        \end{pmatrix} \qquad \mathbf{y} = \begin{bmatrix} y^{(1)} \\ y^{(2)} \\ \vdots \\ y^{(m)} \\\end{bmatrix}$$

벡터화 된 버전은 'numpy'와 같은 수치 계산 도구로 작업 하게 된다면 효율적으로 수행할 수 있습니다. 당신이 행렬 연산을 쫌 한다면 두 형식이 동일하다는 것을 알 수 있을 것입니다.
</div>

<a id="computeCostMulti"></a>

In [ ]:
def computeCostMulti(X, y, theta):
    """
    여러 변수를 사용한 선형 회귀 분석 비용 함수를 계산하세요.
    데이터 포인트 i에 맞게 선형 회귀 매개 변수로 theta를 사용하는 비용을 계산합니다
    
    [Parameters]
    ----------
    X : array 형태 (m x n+1).
    
    y : array 형태 (m, )
    
    theta : array 형태 (n+1, ). 선형 회귀 parameters.
    
    
    [Returns]
    -------
    J : float, 비용 함수의 값
    
    [Instructions]
    ------------
    선택한 세타의 비용을 계산합니다. J를 비용으로 설정해야합니다.
    """
    # 사용할 values를 초기화 합시다
    m = y.shape[0] # training examples의 개수
    
    # 다음 변수를 올바르게 반환해야합니다.
    # ======================= YOUR CODE HERE ===========================
    
    J = None
    
    # ==================================================================
    return J


*정답을 제출하세요*

In [ ]:
grader.answer[1] = computeCostMulti
grader.grade(1)

<a id="gradientDescentMulti"></a>

$$
\theta_0 := \theta_0 - \alpha \frac{1}{m}\sum_{i=1}^m (h_\theta (x^{(i)}) - y^{(i)}) \cdot x_0^{(i)} \\
\theta_1 := \theta_1 - \alpha \frac{1}{m}\sum_{i=1}^m (h_\theta (x^{(i)}) - y^{(i)}) \cdot x_1^{(i)} \\
\theta_2 := \theta_2 - \alpha \frac{1}{m}\sum_{i=1}^m (h_\theta (x^{(i)}) - y^{(i)}) \cdot x_2^{(i)} \\
$$

In [ ]:
def gradientDescentMulti(X, y, theta, alpha, num_iters):
    """
    세타를 학습시키기 위해 경사 하강을 수행합니다.
    learning rate인 alpha로 num_iters 그레디언트 단계를 수행하여 theta를 업데이트합니다.
        
    [Parameters]
    ----------
    X : array 형식 data set (m x n+1).
    
    y : array 형식. 주어진 데이터 포인트의 값에 대한 모양 (m,)의 벡터
    
    theta : array 형식의 선형 회귀에 대한 변수. (n+1, )
    
    alpha : float 형식. 경사하강에서의 learning rate
    
    num_iters : int 형식. 경사하강에서 시행 횟수

    
    [Returns]
    -------
    theta : array 형식 (n+1, ) 선형 회귀로부터 학습된 변수
     
    J_history : list 형식. 매 시행 횟수마다의 비용함수의 값이 있음

    
    [Instructions]
    ------------
    모수 벡터 theta에서 단일 그래디언트 단계를 수행하세요.
    디버깅하는 동안 여기에서 비용 함수 (계산 비용) 및 그라디언트의 값을 인쇄해 봐도 좋을 것 같아요.
    """
    # 유용한 value 초기화
    m = y.shape[0] # training examples 갯수
    
    # 경사 하강에 쓰일 theta를 복사해두기
    theta = theta.copy()
    
    J_history = []
    
    for i in range(num_iters):
        # ======================= YOUR CODE HERE ==========================

        
        # =================================================================
        
        # 매 시행 횟수마다 J를 저장
        J_history.append(computeCostMulti(X, y, theta))
    
    return theta, J_history

*정답을 제출하세요*

In [ ]:
grader.answer[2] = gradientDescentMulti
grader.grade(2)

#### Learning rates 선택하기

이 연습에서는 데이터 집합에 대해 다른 학습 속도를 시도하고 빠르게 수렴하는 학습 속도를 찾습니다. 다음 코드를 수정하고 학습 속도를 설정하는 코드 부분을 변경하여 학습 속도를 변경할 수 있습니다.

`gradientDescentMulti` 함수 구현을 사용하고 선택한 학습 속도로 약 50 회 반복 그라디언트 디센트를 실행하세요. 이 함수는 벡터 $ J $에 $ J (\ theta) $ 값의 이력을 반환해야합니다.

마지막 반복 후 반복 횟수에 대해 J 값을 플로팅합니다.

학습 범위 내에서 학습률을 선택한 경우 플롯 그림은 다음 그림과 유사합니다.

![](Figures/learning_rate.png)

그래프가 매우 다르게 보이면, 특히 $ J (\ theta) $ 값이 증가하거나 폭증하는 경우 학습 속도를 조정하고 다시 시도하십시오. 학습률 $ \ alpha $의 값을 이전 값의 약 3 배 (즉, 0.3, 0.1, 0.03, 0.01 등)의 곱셈 단계에서 로그 스케일로 시도하는 것을 추천드립니다. 곡선에서 전체 추세를 확인하고 나서 실행중인 반복 횟수를 조정할 수도 있습니다.

<div class="alert alert-block alert-warning">
**Implementation Note:** 학습률이 너무 높으면 $ J (\ theta) $가 발산되어 너무 큰 값을 초래할 수 있습니다. 이 경우 numpy에서 NaN이 출력되는 경우가 발생합니다. NaN은 숫자가 아님을 나타내며 종종 -∞ 및 + ∞을 포함하는 정의되지 않은 연산으로 인해 발생합니다.
</div>


학습률이 변함에 따라 수렴 곡선의 변화에 주목하세요. 학습률이 낮으면 GD가 최적의 값으로 수렴하는 데 시간이 오래 걸립니다. 반대로 학습률이 높으면 경사 하강이 수렴하지 않거나 발산 될 수도 있습니다. 찾은 최고의 학습률을 사용하여 스크립트를 실행하세요. 수렴 할 때까지 기울기 하강을 실행하여 $ \ theta $의 최종 값을 찾습니다. 다음에는 이 $ \ theta $ 값을 사용하여 1650 제곱 피트, 침실 3 개인 주택의 값을 예상해보세요. 나중에 값을 사용하여 정규 방정식의 구현을 확인합니다. 이 예측을 할 때 함수를 정규화하는 것을 잊지 마세요!

In [ ]:
"""
[Instructions]
------------
특정 학습 속도 (alpha)로 GD를 실행하는 코드의 시작을 제공해 드렸습니다.
이제 당신의 임무는 먼저 함수-computeCost와 gradientDescent가
이 시작 코드와 함께 작동하는지 확인하고, 다변량 회귀를 확인하는 것입니다.
그 후, 다른 alpha 값으로 GD를 실행하고 어느 것이 가장 좋은 결과를 가져오는지 확인하세요.
마지막으로 코드를 완성하여 1650 평방 피트, 3br 주택의 가격을 예측해보세요!

[Hint]
----
예측할 때 feature 정규화를 적용해야 합니다!
"""
# 이 부분을 바꿔서 알파 값을 선택하세요
alpha = 0.1
num_iters = 400

# theta를 초기화하고 GD 실행
theta = np.zeros(3)
theta, J_history = gradientDescentMulti(X, y, theta, alpha, num_iters)

# 수렴 그래프 그리기
pyplot.plot(np.arange(len(J_history)), J_history, lw=2)
pyplot.xlabel('Number of iterations')
pyplot.ylabel('Cost J')

# GD 결과 표시
print('theta computed from gradient descent: {:s}'.format(str(theta)))

#  1650 평방 피트, 3br 주택의 가격을 예측해보세요
# ======================= YOUR CODE HERE ===========================
# X의 첫 번째 열은 모두 1이라는 것을 기억하세요!
# 따라서 정규화 할 필요가 없습니다.

price = None   # 이 부분을 바꾸세요

# ===================================================================

print('Predicted price of a 1650 sq-ft, 3 br house (using gradient descent): ${:.0f}'.format(price))

이 부분에 대한 답안을 제출하지 않아도됩니다.

<a id="section7"></a>
### Normal Equations(정규방정식)

앤드류 응 형님의 강의에서 선형 회귀에 대한 수학적 표현식은 다음과 같습니다.

$$ \theta = \left( X^T X\right)^{-1} X^T\vec{y}$$

이 공식을 사용하면 피쳐 스케일링이 필요하지 않으며 한 번의 계산으로 정확한 솔루션을 얻을 수 있습니다. 그래디언트 디센트와 같이 수렴까지 루프가 없습니다. 먼저 변수를 수정하지 않도록 데이터를 다시 로드합니다. feature을 확장 할 필요는 없지만 절편 ($ \theta_0 $)를 사용하려면 $ X $ 행렬에 1의 열을 추가해야합니다. 다음 셀의 코드는 1의 열을 X에 추가합니다.

`np.linalg.pinv (또는 inv)` 를 통해 역행렬을 구할 수 있습니다. pinv는 입력된 행렬의 역행렬이 없을 경우 의사역행렬(pseudo inverse matrix)를 출력합니다.

In [ ]:
# 데이터 불러오기
data = np.loadtxt(os.path.join('Data', 'ex1data2.txt'), delimiter=',')
X = data[:, :2]
y = data[:, 2]
m = y.size
X = np.concatenate([np.ones((m, 1)), X], axis=1)

위의 공식을 사용하여 $ \ theta $를 계산하려면 아래 `normalEqn` 함수의 코드를 완성하십시오.
<a id="normalEqn"></a>

In [ ]:
def normalEqn(X, y):
    """
    정규 방정식을 사용하여 수학적 표현식을 선형 회귀로 계산합니다.
    
    [Parameters]
    ----------
    X : array 형태 (m x n+1).
    
    y : array 형태 (m, ).
    
    [Returns]
    -------
    theta : array 형태(n+1, ) 선형 회귀의 계수
    
    [Instructions]
    ------------
    선형 회귀에 대한 수학적 표현식을 계산하고, 그 결과를 세타에 넣으려는 코드를 완성하십시오.
    
    [Hint]
    ----
    역행렬을 계산하기 위해 함수 'np.linalg.pinv'를 사용해보세요.
    """
    
    # ===================== YOUR CODE HERE ============================
    
    theta = None
    
    # =================================================================
    return theta

*정답을 제출하세요*

In [ ]:
grader.answer[3] = normalEqn
grader.grade(3)

이제 이 방법을 사용하여 $ \ theta $를 찾은 후 3 개의 침실이있는 1650 평방 피트의 주택 가격을 예측하는 데 사용하십시오. GD에 맞는 모델을 사용하여 얻은 값과 동일한 예측 가격을 제공하는 것을 알 수 있습니다.

In [ ]:
# 정규 방정식에서 parameter를 계산
theta = normalEqn(X, y);

# 정규 방정식 결과 표시
print('Theta computed from the normal equations: {:s}'.format(str(theta)));

# 1650 sq-ft, 3 br 주택의 가격을 추정하기
# ====================== YOUR CODE HERE ======================

price = 0 # 여기를 바꿔보세요


# ============================================================

print('Predicted price of a 1650 sq-ft, 3 br house (using normal equations): ${:.0f}'.format(price))